### 'Assault', 'Robbery', and 'Sex Offences Forcible' categories. Let's call them 'Violent'
![](./img/img1.jpeg)
![](./img/img2.jpeg)
![](./img/img3.jpeg)
![](./img/img4.jpeg)
![](./img/img5.jpeg)
![](./img/img6.jpeg)

### This time, taking into account all the categories of crime
![](./img/img7.jpeg)

### добавить адрес
### как-то разбить Address, чтоб было меньше чем 23000 уникальных эл-ов
### если убрать Category, то множество строк будут одинаковыми, как бороться?
### можно добавить polynomial features от scikit

In [1]:
import pandas as pd
import numpy as np
import cvxopt
import cvxopt.solvers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from kernels import Kernel
import svm
import time

In [10]:
import pickle

In [6]:
train = pd.read_csv('/Users/grigoriipogorelov/Desktop/train.csv')
train['Date'] = pd.to_datetime(train['Dates'], errors='coerce')
train['Year'] = train['Date'].dt.year
train['Month'] = train['Date'].dt.month
train['WeekOfYear'] = train['Date'].dt.weekofyear
train['Day'] = train['Date'].dt.day
train['Time'] = train['Date'].dt.time
train['Hour'] = train['Date'].dt.hour
train = train.drop(['Dates','Date','Time','Resolution','Descript','Address'], axis=1)


test = pd.read_csv('/Users/grigoriipogorelov/Desktop/test.csv')
test['Date'] = pd.to_datetime(test['Dates'], errors='coerce')
test['Year'] = test['Date'].dt.year
test['Month'] = test['Date'].dt.month
test['WeekOfYear'] = test['Date'].dt.weekofyear
test['Day'] = test['Date'].dt.day
test['Time'] = test['Date'].dt.time
test['Hour'] = test['Date'].dt.hour
test = test.drop(['Dates','Date','Time','Address'], axis=1)

,Category,DayOfWeek,PdDistrict,X,Y,Year,Month,WeekOfYear,Day,Hour
0,WARRANTS,Wednesday,NORTHERN,-122.425892,37.774599,2015,5,20,13,23
1,OTHER OFFENSES,Wednesday,NORTHERN,-122.425892,37.774599,2015,5,20,13,23
2,OTHER OFFENSES,Wednesday,NORTHERN,-122.424363,37.800414,2015,5,20,13,23
3,LARCENY/THEFT,Wednesday,NORTHERN,-122.426995,37.800873,2015,5,20,13,23
4,LARCENY/THEFT,Wednesday,PARK,-122.438738,37.771541,2015,5,20,13,23


In [7]:
cat_cols = ['DayOfWeek','PdDistrict', 'Year', 'Month', 'WeekOfYear', 'Day', 'Hour']
for cat in cat_cols:
    train = pd.get_dummies(train, columns=[cat])
    test = pd.get_dummies(test, columns=[cat])

    
train_proportion = 0.006
train = train.sample(frac=1)
train = train.iloc[:int(len(train)*train_proportion)]
# test = train.iloc[int(len(train)*train_proportion):2*int(len(train)*train_proportion)]


categories = train.Category.unique()
category_len = len(train['Category'].unique())
samples = []
for i in range(category_len-1):
    for j in range(i+1, category_len):
        samples.append(categories[i] + '+' + categories[j])
        

train_sets = []
for i in range(len(samples)):
    cats = samples[i].split('+')
    train_0 = train.loc[train['Category'] == cats[0]]
    train_1 = train.loc[train['Category'] == cats[1]]
    frames = [train_0, train_1]
    train_sets.append(pd.concat(frames))

In [8]:
len(train_sets)

630

In [9]:
start = time.time()

models = []
C = 0.1
for data in train_sets:
    options = data['Category'].unique()
    data = data.as_matrix()
    data[data == options[0]] = 1
    data[data == options[1]] = -1
    data = np.array(data, dtype='float')
    trainer = svm.SVMTrainer(kernel=Kernel.linear(), c=C)
    model = trainer.train(data[:,1:], data[:,0])
    models.append(model)

end = time.time()
print((end - start)/60, ' minutes')

     pcost       dcost       gap    pres   dres
 0: -5.6390e+01 -2.1205e+02  8e+03  3e+01  3e-10
 1: -7.8638e+00 -1.9369e+02  4e+02  7e-01  3e-10
 2: -5.5141e+00 -4.0909e+01  4e+01  3e-02  5e-11
 3: -5.4209e+00 -1.0832e+01  6e+00  4e-03  4e-11
 4: -5.5352e+00 -7.7977e+00  2e+00  4e-15  5e-11
 5: -5.6752e+00 -6.3281e+00  7e-01  5e-15  5e-11
 6: -5.7438e+00 -6.0085e+00  3e-01  4e-15  4e-11
 7: -5.7745e+00 -5.8652e+00  9e-02  2e-16  5e-11
 8: -5.7874e+00 -5.8200e+00  3e-02  3e-15  5e-11
 9: -5.7928e+00 -5.8045e+00  1e-02  2e-15  4e-11
10: -5.7949e+00 -5.7992e+00  4e-03  5e-15  4e-11
11: -5.7960e+00 -5.7970e+00  1e-03  4e-15  5e-11
12: -5.7963e+00 -5.7966e+00  3e-04  7e-15  4e-11
13: -5.7964e+00 -5.7965e+00  4e-05  9e-15  5e-11
14: -5.7964e+00 -5.7964e+00  9e-07  1e-15  5e-11
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -4.7217e+01 -9.5385e+01  4e+03  3e+01  2e-10
 1: -7.6352e+00 -8.7640e+01  2e+02  9e-01  2e-10
 2: -5.2570e+00 -2.4119e+01  2e+01  9e-03  4e-1

     pcost       dcost       gap    pres   dres
 0: -3.2469e+01 -4.6179e+01  1e+03  3e+01  9e-11
 1: -7.5597e+00 -4.2445e+01  1e+02  1e+00  8e-11
 2: -5.0609e+00 -1.7743e+01  1e+01  4e-15  3e-11
 3: -5.2662e+00 -7.6553e+00  2e+00  1e-15  2e-11
 4: -5.4391e+00 -6.1506e+00  7e-01  1e-15  2e-11
 5: -5.5284e+00 -5.7563e+00  2e-01  7e-16  2e-11
 6: -5.5701e+00 -5.6273e+00  6e-02  3e-16  2e-11
 7: -5.5850e+00 -5.5901e+00  5e-03  8e-16  2e-11
 8: -5.5867e+00 -5.5871e+00  4e-04  6e-16  2e-11
 9: -5.5868e+00 -5.5868e+00  2e-05  5e-16  2e-11
10: -5.5868e+00 -5.5868e+00  4e-07  2e-15  2e-11
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -3.7239e+01 -5.9022e+01  2e+03  3e+01  1e-10
 1: -7.3476e+00 -5.4441e+01  1e+02  1e+00  1e-10
 2: -5.0468e+00 -1.9076e+01  1e+01  5e-16  3e-11
 3: -5.2473e+00 -7.9635e+00  3e+00  9e-16  2e-11
 4: -5.4821e+00 -5.9927e+00  5e-01  1e-15  2e-11
 5: -5.5740e+00 -5.7188e+00  1e-01  2e-16  2e-11
 6: -5.6076e+00 -5.6308e+00  2e-02  2e-16  2e-1

 6: -1.7570e-01 -1.7617e-01  5e-04  8e-17  2e-13
 7: -1.7588e-01 -1.7591e-01  2e-05  1e-16  3e-13
 8: -1.7589e-01 -1.7589e-01  9e-07  1e-16  3e-13
 9: -1.7589e-01 -1.7589e-01  8e-08  1e-16  2e-13
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -5.9128e+02 -3.6439e+02  1e+04  3e+01  4e-09
 1: -9.2436e+01 -3.4803e+02  6e+02  9e-01  3e-09
 2: -7.7176e+01 -1.5888e+02  8e+01  8e-14  6e-10
 3: -8.5024e+01 -1.0543e+02  2e+01  4e-14  5e-10
 4: -8.7633e+01 -9.5682e+01  8e+00  4e-14  6e-10
 5: -8.8913e+01 -9.1124e+01  2e+00  3e-14  6e-10
 6: -8.9339e+01 -8.9936e+01  6e-01  6e-14  6e-10
 7: -8.9469e+01 -8.9670e+01  2e-01  9e-14  6e-10
 8: -8.9507e+01 -8.9600e+01  9e-02  5e-15  6e-10
 9: -8.9525e+01 -8.9572e+01  5e-02  1e-13  6e-10
10: -8.9536e+01 -8.9555e+01  2e-02  5e-15  6e-10
11: -8.9541e+01 -8.9547e+01  7e-03  8e-14  6e-10
12: -8.9543e+01 -8.9544e+01  2e-03  9e-14  6e-10
13: -8.9543e+01 -8.9544e+01  4e-04  1e-13  6e-10
14: -8.9543e+01 -8.9543e+01  7e-05  6e-14  6e-

 2: -1.0605e+01 -5.5366e+01  5e+01  3e-02  7e-11
 3: -1.0617e+01 -2.1293e+01  1e+01  6e-03  5e-11
 4: -1.0874e+01 -1.3386e+01  3e+00  6e-15  6e-11
 5: -1.1095e+01 -1.1751e+01  7e-01  6e-15  6e-11
 6: -1.1166e+01 -1.1378e+01  2e-01  2e-15  6e-11
 7: -1.1187e+01 -1.1270e+01  8e-02  9e-15  6e-11
 8: -1.1196e+01 -1.1220e+01  2e-02  7e-15  5e-11
 9: -1.1199e+01 -1.1204e+01  5e-03  4e-15  6e-11
10: -1.1200e+01 -1.1201e+01  2e-03  7e-15  6e-11
11: -1.1200e+01 -1.1201e+01  8e-04  8e-15  5e-11
12: -1.1200e+01 -1.1200e+01  3e-04  6e-16  6e-11
13: -1.1200e+01 -1.1200e+01  1e-04  5e-15  6e-11
14: -1.1200e+01 -1.1200e+01  5e-05  2e-15  6e-11
15: -1.1200e+01 -1.1200e+01  2e-05  2e-14  6e-11
16: -1.1200e+01 -1.1200e+01  2e-05  1e-14  5e-11
17: -1.1200e+01 -1.1200e+01  7e-06  5e-15  5e-11
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -4.6315e+01 -2.0714e+02  8e+03  3e+01  2e-10
 1: -6.7867e+00 -1.8881e+02  4e+02  8e-01  2e-10
 2: -4.6854e+00 -4.3007e+01  4e+01  4e-02  3e-

 3: -6.9185e+00 -1.5611e+01  9e+00  4e-03  3e-11
 4: -7.0614e+00 -9.8695e+00  3e+00  1e-15  3e-11
 5: -7.2578e+00 -7.9723e+00  7e-01  2e-15  3e-11
 6: -7.3267e+00 -7.6832e+00  4e-01  3e-15  3e-11
 7: -7.3668e+00 -7.4854e+00  1e-01  2e-15  3e-11
 8: -7.3858e+00 -7.4190e+00  3e-02  2e-15  3e-11
 9: -7.3925e+00 -7.4009e+00  8e-03  2e-15  3e-11
10: -7.3944e+00 -7.3974e+00  3e-03  5e-15  3e-11
11: -7.3949e+00 -7.3966e+00  2e-03  7e-15  3e-11
12: -7.3955e+00 -7.3957e+00  3e-04  6e-15  3e-11
13: -7.3956e+00 -7.3956e+00  3e-05  1e-15  3e-11
14: -7.3956e+00 -7.3956e+00  2e-06  8e-16  3e-11
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -9.2903e+01 -2.2162e+02  9e+03  3e+01  4e-10
 1: -1.4660e+01 -2.0359e+02  4e+02  9e-01  3e-10
 2: -1.0433e+01 -5.6815e+01  5e+01  5e-02  7e-11
 3: -1.0330e+01 -2.3741e+01  1e+01  1e-02  6e-11
 4: -1.0787e+01 -1.4254e+01  4e+00  2e-03  6e-11
 5: -1.1084e+01 -1.2553e+01  1e+00  1e-14  6e-11
 6: -1.1290e+01 -1.1789e+01  5e-01  6e-15  5e-

...

     pcost       dcost       gap    pres   dres
 0: -4.0155e-01 -3.1957e+00  8e+01  8e+00  8e-13
 1: -2.9366e-01 -2.7693e+00  9e+00  7e-01  5e-13
 2: -1.6293e-01 -1.0571e+00  9e-01  2e-16  4e-13
 3: -1.6833e-01 -2.8274e-01  1e-01  8e-17  2e-13
 4: -1.7375e-01 -1.9785e-01  2e-02  9e-17  2e-13
 5: -1.7552e-01 -1.7898e-01  3e-03  1e-16  2e-13
 6: -1.7628e-01 -1.7655e-01  3e-04  7e-17  2e-13
 7: -1.7641e-01 -1.7643e-01  2e-05  6e-17  2e-13
 8: -1.7643e-01 -1.7643e-01  1e-06  7e-17  2e-13
 9: -1.7643e-01 -1.7643e-01  8e-08  9e-17  3e-13
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -3.8068e-01 -3.1730e+00  8e+01  8e+00  4e-13
 1: -2.9393e-01 -2.7511e+00  9e+00  8e-01  5e-13
 2: -1.6272e-01 -1.0712e+00  9e-01  1e-16  4e-13
 3: -1.6829e-01 -2.7190e-01  1e-01  7e-17  2e-13
 4: -1.7298e-01 -1.9435e-01  2e-02  7e-17  2e-13
 5: -1.7424e-01 -1.7697e-01  3e-03  9e-17  1e-13
 6: -1.7470e-01 -1.7503e-01  3e-04  8e-17  1e-13
 7: -1.7480e-01 -1.7483e-01  3e-05  6e-17  2e-1

 7: -3.5461e-01 -3.5464e-01  3e-05  1e-16  4e-13
 8: -3.5462e-01 -3.5462e-01  1e-06  1e-16  4e-13
 9: -3.5462e-01 -3.5462e-01  2e-08  1e-16  5e-13
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -8.3173e-01 -5.3303e+00  1e+02  1e+01  1e-12
 1: -5.7903e-01 -4.6248e+00  1e+01  9e-01  1e-12
 2: -3.2684e-01 -1.8183e+00  1e+00  2e-16  6e-13
 3: -3.3723e-01 -5.2197e-01  2e-01  1e-16  4e-13
 4: -3.4647e-01 -3.9508e-01  5e-02  1e-16  4e-13
 5: -3.4944e-01 -3.5552e-01  6e-03  1e-16  5e-13
 6: -3.5072e-01 -3.5136e-01  6e-04  1e-16  4e-13
 7: -3.5094e-01 -3.5098e-01  4e-05  1e-16  4e-13
 8: -3.5096e-01 -3.5096e-01  1e-06  1e-16  4e-13
 9: -3.5096e-01 -3.5096e-01  7e-08  1e-16  4e-13
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -8.7197e-01 -5.4428e+00  1e+02  1e+01  2e-12
 1: -5.8161e-01 -4.7218e+00  1e+01  9e-01  1e-12
 2: -3.2635e-01 -1.8207e+00  1e+00  1e-16  6e-13
 3: -3.3708e-01 -5.2820e-01  2e-01  9e-17  4e-13
 4: -3.4728e-01 -3.7569

     pcost       dcost       gap    pres   dres
 0: -6.2491e+00 -1.7803e+01  5e+02  2e+01  1e-11
 1: -2.9931e+00 -1.5722e+01  4e+01  1e+00  1e-11
 2: -1.9290e+00 -6.6750e+00  5e+00  5e-16  8e-12
 3: -2.0043e+00 -2.8268e+00  8e-01  2e-16  6e-12
 4: -2.0766e+00 -2.2602e+00  2e-01  2e-16  6e-12
 5: -2.1056e+00 -2.1388e+00  3e-02  2e-16  5e-12
 6: -2.1134e+00 -2.1175e+00  4e-03  3e-16  5e-12
 7: -2.1147e+00 -2.1149e+00  2e-04  5e-16  6e-12
 8: -2.1147e+00 -2.1148e+00  8e-06  2e-16  5e-12
 9: -2.1148e+00 -2.1148e+00  3e-07  2e-16  5e-12
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -2.9221e+00 -6.2614e+00  1e+02  1e+01  4e-12
 1: -1.9135e+00 -5.5516e+00  8e+00  5e-01  5e-12
 2: -1.6186e+00 -2.7342e+00  1e+00  2e-16  3e-12
 3: -1.6971e+00 -1.8090e+00  1e-01  3e-16  3e-12
 4: -1.7311e+00 -1.7403e+00  9e-03  9e-17  3e-12
 5: -1.7342e+00 -1.7346e+00  4e-04  3e-16  3e-12
 6: -1.7344e+00 -1.7344e+00  1e-05  5e-16  3e-12
 7: -1.7344e+00 -1.7344e+00  1e-06  3e-16  3e-1

     pcost       dcost       gap    pres   dres
 0: -1.3589e+01 -2.1309e+01  5e+02  2e+01  2e-11
 1: -5.1340e+00 -1.8855e+01  4e+01  1e+00  2e-11
 2: -3.4536e+00 -9.2796e+00  6e+00  1e-15  1e-11
 3: -3.6180e+00 -4.6708e+00  1e+00  2e-16  8e-12
 4: -3.7642e+00 -3.9690e+00  2e-01  4e-16  7e-12
 5: -3.8130e+00 -3.8354e+00  2e-02  8e-16  7e-12
 6: -3.8203e+00 -3.8220e+00  2e-03  8e-16  7e-12
 7: -3.8210e+00 -3.8210e+00  4e-05  1e-15  8e-12
 8: -3.8210e+00 -3.8210e+00  1e-06  9e-16  7e-12
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -3.1247e+00 -1.5817e+01  5e+02  2e+01  4e-12
 1: -1.4013e+00 -1.3970e+01  4e+01  1e+00  4e-12
 2: -8.2216e-01 -5.0091e+00  4e+00  3e-16  2e-12
 3: -8.4579e-01 -1.6197e+00  8e-01  2e-16  1e-12
 4: -8.7261e-01 -9.9753e-01  1e-01  2e-16  1e-12
 5: -8.8718e-01 -9.1183e-01  2e-02  2e-16  1e-12
 6: -8.9275e-01 -8.9699e-01  4e-03  2e-16  1e-12
 7: -8.9401e-01 -8.9419e-01  2e-04  2e-16  1e-12
 8: -8.9409e-01 -8.9410e-01  5e-06  6e-16  1e-1

Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -4.3665e-01 -1.1428e+00  2e+01  4e+00  7e-13
 1: -3.8497e-01 -1.0071e+00  1e+00  2e-01  5e-13
 2: -3.0184e-01 -4.9178e-01  2e-01  7e-17  3e-13
 3: -3.0913e-01 -3.2141e-01  1e-02  5e-17  3e-13
 4: -3.1149e-01 -3.1210e-01  6e-04  6e-17  4e-13
 5: -3.1175e-01 -3.1179e-01  4e-05  3e-17  4e-13
 6: -3.1178e-01 -3.1178e-01  9e-07  3e-17  4e-13
 7: -3.1178e-01 -3.1178e-01  9e-09  5e-17  4e-13
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -4.7470e-01 -1.1797e+00  2e+01  4e+00  6e-13
 1: -4.1058e-01 -1.0324e+00  2e+00  2e-01  6e-13
 2: -3.0896e-01 -5.2345e-01  2e-01  5e-17  6e-13
 3: -3.1683e-01 -3.3325e-01  2e-02  5e-17  4e-13
 4: -3.1862e-01 -3.1943e-01  8e-04  4e-17  4e-13
 5: -3.1868e-01 -3.1869e-01  1e-05  4e-17  5e-13
 6: -3.1868e-01 -3.1868e-01  1e-07  6e-17  4e-13
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -2.4923e-01 -8.8785e-01  2e+01  4e+00  6e-13
